# Series Collection

Often users of the FRED API will want analyze multiple economic series. This can be done with `FredSeries` alone, but can be tedious and cumbersome. `pyfredapi` offers the `SeriesCollection` class to streamline the process of collecting and munging data for plotting and analysis.

A `SeriesCollection` object is a set of `SeriesData` objects. `SeriesCollection` provide helpful methods to:

* List the metadata (frequency, seasonality, units, etc.) of the series in the collection
* Merge series dataframes into a long dataframe
* Merge series dataframes into a wide dataframe by index
* Merge series dataframes into a wide dataframe by date

## Setup

Import pyfredapi

In [1]:
import pyfredapi as pf
from rich.pretty import pprint

## Create a SeriesCollection

Create an instance of `SeriesCollection`,

Add data to the collection with `add_series()`. By default the column for the series values will be renamed to the series id.

In [2]:
sc = pf.SeriesCollection(series_ids=["GDP"])

Requesting series GDP...


## Collect additional series

Add more series to a `SeriesCollection` object with `add()`.

In [3]:
sc.add(series_ids=["SP500"])

Requesting series SP500...


### Remove series

Remove series from the collection with `remove()`.

In [ ]:
sc.remove("SP500")

In [ ]:
del sc["SP500"]

## Plot Series

The `plot` method builds a [plotly](https://plotly.com/python/) time series plot of the data.

In [ ]:
fig = sc.GDP.plot()
fig.show(renderer="notebook")

## Accessing the data

The `SeriesCollection` is composed of `SeriesData` objects. You can access the `SeriesData` by attribute. Each series_id added to the collection will be an attribute that returns the `SeriesData` object for that series.

`SeriesData` is has two attributes.

* `info` - The series metadata.
* `df` - Series observations in a pandas dataframe.

### Access via attribute

In [8]:
sc.GDP == sc["GDP"]

True

In [6]:
pprint(sc.GDP.info)

SeriesInfo(
│   id='GDP',
│   realtime_start='2023-04-27',
│   realtime_end='2023-04-27',
│   title='Gross Domestic Product',
│   observation_start='1947-01-01',
│   observation_end='2023-01-01',
│   frequency='Quarterly',
│   frequency_short='Q',
│   units='Billions of Dollars',
│   units_short='Bil. of $',
│   seasonal_adjustment='Seasonally Adjusted Annual Rate',
│   seasonal_adjustment_short='SAAR',
│   last_updated='2023-04-27 07:53:02-05',
│   popularity=93,
│   notes='BEA Account Code: A191RC\n\nGross domestic product (GDP), the featured measure of U.S. output, is the market value of the goods and services produced by labor and property located in the United States.For more information, see the Guide to the National Income and Product Accounts of the United States (NIPA) and the Bureau of Economic Analysis (http://www.bea.gov/national/pdf/nipaguid.pdf).'
)

In [ ]:
sc.GDP.df.tail()

## Access via bracket notation

In [7]:
pprint(sc["GDP"].info)

SeriesInfo(
│   id='GDP',
│   realtime_start='2023-04-27',
│   realtime_end='2023-04-27',
│   title='Gross Domestic Product',
│   observation_start='1947-01-01',
│   observation_end='2023-01-01',
│   frequency='Quarterly',
│   frequency_short='Q',
│   units='Billions of Dollars',
│   units_short='Bil. of $',
│   seasonal_adjustment='Seasonally Adjusted Annual Rate',
│   seasonal_adjustment_short='SAAR',
│   last_updated='2023-04-27 07:53:02-05',
│   popularity=93,
│   notes='BEA Account Code: A191RC\n\nGross domestic product (GDP), the featured measure of U.S. output, is the market value of the goods and services produced by labor and property located in the United States.For more information, see the Guide to the National Income and Product Accounts of the United States (NIPA) and the Bureau of Economic Analysis (http://www.bea.gov/national/pdf/nipaguid.pdf).'
)

## Rename series in the collection

### Rename on add

You can rename the series when adding them to the collection. Renaming can be done with a dictionary mapping the series id to the new name, or with a function which parses the series title into the new name.

In [ ]:
# Rename with a dictionary
new_names = {
    "CPIAUCSL": "cpi_all_items",
    "CPILFESL": "cpi_all_items_less_food_and_energy",
}

cpi_sc = pf.SeriesCollection(series_ids=["CPIAUCSL", "CPILFESL"], rename=new_names)

In [ ]:
cpi_sc.CPIAUCSL.df.head()

In [ ]:
cpi_sc.CPILFESL.df.head()

### Rename after add

You can rename series in the collection with the `rename_series` method. Works the same way as renaming on add.

In [ ]:
def parse_cpi_title(title: str) -> str:
    """Parse CPI series title into a readable label."""
    return (
        title.lower()
        .replace("consumer price index", "CPI ")
        .replace(" for all urban consumers: ", "")
        .replace(" in u.s. city average", "")
        .title()
    )


cpi_sc.rename_series(rename=parse_cpi_title)

In [ ]:
cpi_sc.CPIAUCSL.df.head()

## List metadata

`SeriesCollection` has a number of list methods to print out the metadata of the series in the collection.

### Series in the collection

In [ ]:
cpi_sc.list_series()

### Frequency

In [ ]:
cpi_sc.list_frequency()

### Seasonality

In [ ]:
cpi_sc.list_seasonality()

### Units

In [ ]:
cpi_sc.list_units()

### Dates

In [ ]:
cpi_sc.list_end_date()

In [ ]:
cpi_sc.list_start_date()

## Merge data

`SeriesCollection` supports merging the data into long and wide formats. By default the series ID will be used as the column name or observation label.

### Merge long

Merge the series in the collection into a long pandas dataframe.

In [ ]:
cpi_long = cpi_sc.merge_long()
cpi_long

### Merge as-of

Merge the series in the collection into a wide pandas dataframe based on nearest date. Must define a base series. The base series defines the set of dates to serve of the basis of joining.

In [ ]:
cpi_asof = cpi_sc.merge_asof(base_series_id="CPIAUCSL")
cpi_asof.tail()

### Merge wide

Merge the series in the collection into a wide pandas dataframe. Only works if all the series in the collection share the same date index.

In [ ]:
cpi_wide = cpi_sc.merge_wide()
cpi_wide.tail()